# Process US County-level Data from NYT

In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read data and get basic county groups

0.0 Report Data

In [2]:
# Read updated CSV file
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.string_})
# Change 'Unknown' fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
# Change 'Unknown' deaths to 0 and death column to int32 type
df_counties_original['deaths'] = df_counties_original['deaths'].replace({np.nan: 0})
df_counties_original = df_counties_original.astype({'deaths': 'int32'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
1346274,2021-05-23,Unknown,Guam,00000,9103,140
78616,2020-04-22,Unknown,Arkansas,00000,113,0
1882821,2021-11-04,Unknown,Maryland,00000,0,50
78728,2020-04-22,Unknown,Colorado,00000,95,0
78739,2020-04-22,Unknown,Connecticut,00000,550,7
...,...,...,...,...,...,...
1407070,2021-06-10,St. Thomas,Virgin Islands,78030,1996,17
440365,2020-08-16,St. Thomas,Virgin Islands,78030,393,4
1556399,2021-07-26,St. Thomas,Virgin Islands,78030,2446,22
1504445,2021-07-10,St. Thomas,Virgin Islands,78030,2217,20


0.1 GeoJSON Data

In [3]:
# Read GeoJson data
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


0.2 Group Data to Counties

In [4]:
# Transform cases/deaths data table to pivot table using fips as main index
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

cases                                              \
date                   2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                                                                               \
date                   2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                        ...     deaths                                   \
date                    ... 2021-10-29 2021-10-30 2021-10-31 2021-11-01   
state   county   fips   ...                                               
Alabama Autauga  01001  ...      148.0      148.0      148.0      148.0   
        Baldwin  01003  ...      555.0      555.0      555.0      555.0   
        Barbour  01005  ...       76.0       76.0       76.0       76.0   
        Bibb     01007  ...       87.0       87.0       87.0       87.0   
        Blount   01009  ...      178.0      178.0      178.0      178.0   
...                     ...        ...        ...        ...        ...   
Wyoming Teton    56039  ...       14.0       14.0       14.0       14.0   
        Uinta    56041  ...       26.0       26.0       26.0       26.0   
        Unknown  00000  ...        0.0        0.0        0.0        0.0   
        Washakie 56043  ...       35.0       35.0       35.0       35.0   
        Weston   56045  ...        9.0        9.0        9.0        9.0   

                                                                               \
date                   2021-11-02 2021-11-03 2021-11-04 2021-11-05 2021-11-06   
state   county   fips                                                           
Alabama Autauga  01001      148.0      150.0      151.0      151.0      151.0   
        Baldwin  01003      558.0      563.0      567.0      574.0      575.0   
        Barbour  01005       76.0       76.0       77.0       79.0       79.0   
        Bibb     01007       89.0       91.0       92.0       92.0       92.0   
        Blount   01009      179.0      181.0      182.0      183.0      186.0   
...                           ...        ...        ...    

0.3 Get Date Info

In [5]:
# Get dates from data table
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01', '2021-11-02',
       '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-06', '2021-11-07'],
      dtype='object', name='date', length=657)

In [6]:
# Sort dates and get the start/end
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [7]:
# Format dates and create a list
start = datetime.strptime(dt_str_start, "%Y-%m-%d")
end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=start,end=end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

657 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01',
               '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05',
               '2021-11-06', '2021-11-07'],
              dtype='datetime64[ns]', length=657, freq='D')
657 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [8]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

# Create time series of cases/deaths
pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()

In [9]:
# Get the date of first case/death
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
# For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths                                   \
date                   ... 2021-11-02 2021-11-03 2021-11-04 2021-11-05   
state   county  fips   ...                                               
Alabama Autauga 01001  ...      148.0      150.0      151.0      151.0   
        Baldwin 01003  ...      558.0      563.0      567.0      574.0   
        Barbour 01005  ...       76.0       76.0       77.0       79.0   
        Bibb    01007  ...       89.0       91.0       92.0       92.0   
        Blount  01009  ...      179.0      181.0      182.0      183.0   

                                             \
date                  2021-11-06 2021-11-07   
state   county  fips                          
Alabama Autauga 01001      151.0      151.0   
        Baldwin 01003      575.0      575.0   
        Barbour 01005       79.0       79.0   
        Bibb    01007       92.0       92.0   
        Blount  01009      186.0      186.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03     2020-04-29  
        Bibb    01007    2020-03-30     2020-05-08  
        Blount  01009    2020-03-25     2020-05-17  

[5 rows x 1318 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [10]:
# Get the latest case/death
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

Step3:  
Extract necessary columns

In [11]:
# Only keep the necessary columns
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-24,2020-04-06,10331.0,4.0,151.0,0.0
1,Alabama,Baldwin,01003,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-14,2020-04-01,37648.0,15.0,575.0,0.0
2,Alabama,Barbour,01005,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-03,2020-04-29,3614.0,2.0,79.0,0.0
3,Alabama,Bibb,01007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-30,2020-05-08,4295.0,1.0,92.0,0.0
4,Alabama,Blount,01009,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-25,2020-05-17,10533.0,6.0,186.0,0.0


Step4:  
Assign geometries to counties

In [12]:
# Merge case dataframe and geometry
final_df = pd.merge(old_counties_geojson_df, report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'GEOID', 'population', 'geometry', 'state',
       'county', 'fips', 'cases_ts', 'deaths_ts', 'dt_first_case',
       'dt_first_death', 'today_case', 'today_new_case', 'today_death',
       'today_new_death'],
      dtype='object')

In [13]:
final_df.head(1)

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",North Carolina,Bladen,37017,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-03,2020-05-06,5499.0,0.0,95.0,0.0


In [14]:
final_df[final_df["cases_ts"].isna()==True]

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
121,Winchester,Virginia,51840,27789,"MULTIPOLYGON (((-78.18779 39.20487, -78.13744 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,Covington,Virginia,51580,5582,"MULTIPOLYGON (((-80.00500 37.76535, -79.98740 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,Staunton,Virginia,51790,24452,"MULTIPOLYGON (((-79.09590 38.18508, -79.03379 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286,Aleutians West,Alaska,02016,5750,"MULTIPOLYGON (((179.48132 51.97530, 179.58286 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,Juana Díaz,Puerto Rico,72075,46960,"MULTIPOLYGON (((-66.54722 18.15313, -66.52190 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,Denali,Alaska,02068,2232,"MULTIPOLYGON (((-152.42808 63.18317, -152.4281...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3073,Alexandria,Virginia,51510,156505,"MULTIPOLYGON (((-77.11080 38.84345, -77.08781 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3080,Lexington,Virginia,51678,7110,"MULTIPOLYGON (((-79.46155 37.78372, -79.45333 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3087,Fairbanks North Star,Alaska,02090,99653,"MULTIPOLYGON (((-148.66120 65.20987, -147.5577...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Step5:  
Deal with nan values

In [18]:
final_df['fips']

0       37017
1       37167
2       39153
3       42113
4       48459
        ...  
3212    31159
3213    37023
3214    13261
3215    00000
3216    00000
Name: fips, Length: 3217, dtype: object

In [17]:
# Replace NAN values with 0
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
#final_df['fips'] = final_df['fips'].replace(np.nan, final_df['GEOID'])

ValueError: Series.replace cannot use dict-value and non-None to_replace

In [19]:
# Create template
template = final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]
template

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 4.0,
 4.0,
 4.0,
 6.0,
 7.0,
 10.0,
 10.0,
 13.0,
 20.0,
 21.0,
 25.0,
 27.0,
 27.0,
 29.0,
 34.0,
 40.0,
 45.0,
 50.0,
 51.0,
 52.0,
 54.0,
 60.0,
 63.0,
 66.0,
 68.0,
 70.0,
 71.0,
 73.0,
 78.0,
 81.0,
 83.0,
 91.0,
 92.0,
 94.0,
 94.0,
 94.0,
 94.0,
 94.0,
 95.0,
 100.0,
 101.0,
 105.0,
 104.0,
 106.0,
 132.0,
 143.0,
 153.0,
 162.0,
 165.0,
 163.0,
 164.0,
 173.0,
 186.0,
 188.0,
 194.0,
 196.0,
 199.0,
 214.0,
 220.0,
 232.0,
 262.0,
 307.0,
 3

In [20]:
# Create string filled with 0s to replace NaN
str0 = "0,"*(len(template)-1) + "0"
str0

'0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [21]:
# Change list format to string with commas
final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else str0)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else str0)
final_df.head(5)

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",North Carolina,Bladen,37017,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-03,2020-05-06,5499.0,0.0,95.0,0.0
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",North Carolina,Stanly,37167,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-04-09,11752.0,0.0,191.0,0.0
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",Ohio,Summit,39153,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-03-27,64072.0,237.0,1144.0,0.0
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",Pennsylvania,Sullivan,42113,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-04,2020-10-14,600.0,1.0,24.0,0.0
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",Texas,Upshur,48459,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-07-25,5856.0,0.0,144.0,0.0


Step6:  
Finalize the dataframe

In [22]:
# Keep and add essential columns
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 
                   'today_case','today_new_case','today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(1)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",5499.0,0.0,95.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2021-11-07,day


In [23]:
# Calculate weekly change rates of cases and form a time-series list
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit,change_ts
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",5499.0,0.0,95.0,0.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",11752.0,0.0,191.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",64072.0,237.0,1144.0,0.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,2020-10-14,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",600.0,1.0,24.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",5856.0,0.0,144.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,Seward,Nebraska,17127,31159,2020-04-05,2020-04-21,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2323.0,0.0,38.0,0.0,"MULTIPOLYGON (((-97.36812 41.04695, -96.91094 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3213,Burke,North Carolina,89712,37023,2020-03-25,2020-04-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",15864.0,0.0,240.0,0.0,"MULTIPOLYGON (((-81.90665 35.88338, -81.94319 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3214,Sumter,Georgia,30352,13261,2020-03-20,2020-03-28,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",4227.0,0.0,135.0,0.0,"MULTIPOLYGON (((-84.43301 32.04196, -84.43121 ...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3215,Kansas City,Missouri,491918,00000,2020-03-20,2020-04-02,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",64244.0,80.0,840.0,0.0,"POLYGON ((-94.44849 38.93168, -94.44869 38.928...",2020-01-21,2021-11-07,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


Step 7:  
Output file

In [24]:
# Save file
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done
